In [1]:
import librosa
import librosa.display
import matplotlib.pyplot as plt

In [2]:
music, sr = librosa.load('./test/bounce_test.wav')

In [12]:
crop_intv = int(60/126/2*sr) # tempo - 126 BPM
crop_intv_2 = int(60/160/2*sr)

In [4]:
test = music[0:crop_intv]

In [5]:
# Make folder before generation test image
# When new testset is saved, its name must append '_' to end like 'name_.jpg'

In [13]:
for i in range(16):
    cropped = music[crop_intv*i:crop_intv*(i+1)]
    cropped = cropped[0:crop_intv_2]
    plt.figure(figsize=(6.898,4.719))
    plt.plot(cropped)
    plt.ylim(-0.7,0.7)
    plt.xlim(0,len(cropped))
    plt.axis('off')
    plt.savefig(fname='./test/images/time/'+str(i)+'_.png', bbox_inches='tight', pad_inches=0)
    plt.close()
    
    
for i in range(16):
    cropped = music[crop_intv*i:crop_intv*(i+1)]
    cropped = cropped[0:crop_intv_2]
    plt.figure(figsize=(6.898,4.719))
    f_cropped=librosa.feature.melspectrogram(cropped)
    librosa.display.specshow(librosa.power_to_db(f_cropped), cmap='jet', y_axis='mel')
    plt.clim(-70,20)
    plt.axis('off')
    plt.savefig(fname='./test/images/mel/'+str(i)+'_.png', bbox_inches='tight', pad_inches=0)
    plt.close()

C:\Users\ADmin\anaconda3\envs\new_torch\lib\site-packages\librosa\display.py:974: MatplotlibDeprecationWarning: The 'basey' parameter of __init__() has been renamed 'base' since Matplotlib 3.3; support for the old name will be dropped two minor releases later.
  scaler(mode, **kwargs)
C:\Users\ADmin\anaconda3\envs\new_torch\lib\site-packages\librosa\display.py:974: MatplotlibDeprecationWarning: The 'linthreshy' parameter of __init__() has been renamed 'linthresh' since Matplotlib 3.3; support for the old name will be dropped two minor releases later.
  scaler(mode, **kwargs)


In [14]:
from drumdataset_load import DrumDataset, DrumDataset_test
from torch.utils.data import DataLoader
import torch
import json

from models import VGG19
from models import ResNet34, ResidualBlock
from models import EfficientNet

In [15]:
with open('./info.json', 'r') as f:
    info = json.load(f)
    
labels = info["labels"]
data_version = info["data_version"]
dataset_t_v_path = info["path"]["t_v_data"][:-1]
#dataset_test_path = info["path"]["t_v_data"][-1]
data_type = info["data_type"]
data_info = info["data_n"]
data_time_path = info["path"]["time_data"]
data_mel_path = info["path"]["mel_data"]

vgg_para = info["hyperparameter"]["vgg"]
resnet_para = info["hyperparameter"]["resnet"]
efficientnet_para = info["hyperparameter"]["efficientnet"]

In [16]:
dataset_test_path = 'test'

In [17]:
test_dataset = DrumDataset_test(labels,dataset_test_path,dtype='mel')
test_loader = DataLoader(test_dataset, batch_size = 1, shuffle = False, num_workers = 0)

In [18]:
save_path="./parameters/parameter/efficientnet_b0_result_mel_aug.pth"

net = EfficientNet.from_name('efficientnet-b0',in_channels=3, num_classes = 23)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.load_state_dict(torch.load(save_path))
net.eval()
net.to(device)

with torch.no_grad():
    for input_test,_ in test_loader:
        input_test = input_test.cuda()
        input_test = input_test.to("cuda").float()
        
        pred_test = net(input_test)
        _, predicted = torch.max(pred_test, 1)
        
        print(labels[predicted.item()])

B+CH
CH
S
B+CH
CH
CH
S+CH
CH
B+CH
CH
S+CH
B
CH
CH
S
CH
